In [3]:
import requests as rq
import time
import pandas as pd
import re


from openpyxl import Workbook
from openpyxl import load_workbook
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as BS

In [4]:
#크롬브라우저 드라이버 경로 설정

path = 'C:\ChromeDriver\chromedriver.exe'
PAUSE_TIME = 1.0

In [5]:
driver = webdriver.Chrome(path)
time.sleep(PAUSE_TIME)

In [6]:
#망고플레이트 긁을 리스트 주소
url = "https://www.mangoplate.com/top_lists/_CH2GRJ"

In [7]:
driver.get(url)
time.sleep(1)

In [16]:
#리스트 더보기 반복 클릭
try:    
    list_add = driver.find_element_by_css_selector('#contents_list > div > button')
except:
    pass

list_num = driver.find_element_by_xpath('/html/body/main/article/header/div/h1')    
lnum = re.findall('\d+', list_num.text)
#print(len(lnum))
lnum = int(lnum[-1])
time.sleep(PAUSE_TIME)


In [17]:
if lnum != 10:
    for a in range(int(lnum/10)):
        try:
            list_add.click()
            time.sleep(PAUSE_TIME)
        except:
            continue

In [18]:
#페이지 내용 긁기
list_text = driver.page_source
l_soup = BS(list_text, 'html.parser')

In [19]:
#링크가져오기
link = l_soup.select('span > a')
index = 0
alink = []
for index in range(lnum):
    #print(link[index].text)
    #print(link[index].get('href'))
    alink.append('https://www.mangoplate.com'+link[index].get('href'))
    #print(alink[index])

In [32]:
for i in range(lnum):
    driver.get(alink[i])
    #리뷰수
    reviewn = driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div/ul/li[1]/button').text
    rnum = re.findall("\d+", reviewn)
    try:
        rn = rnum[len(rnum)-1]
    except IndexError as e:
        rn = rnum
        pass

    #print(rn)
    #===============================================================================
    #더보기
    n = int(rn)/5
    i=0
    try:
        add_show = driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/button')
        for i in range(int(n)):
            add_show.click()
            time.sleep(PAUSE_TIME)
    except:
        pass
    
    time.sleep(PAUSE_TIME)
    text = driver.page_source
    soup = BS(text, 'html.parser')
    #날짜 긁기
    date = soup.find_all('span', 'past-time ng-binding')
    #리뷰 긁기
    review = soup.findAll('span', 'short_review more_review_bind review_content') + soup.find_all('span', 'review_content ng-binding')
    
    index = 0
    index1 = 0
    for i in review:
        index +=1
    for i in date:
        index1 += 1
    
    #엑셀 시작
    wb = Workbook()

    spring = ['03','04','05']
    summer = ['06', '07','08']
    fall = ['09', '10', '11']
    winter = ['12','01','02']

    sheet1 = wb.active
    sheet1.title = "봄"
    sheet1.cell(1, column = 1).value = '날짜'
    sheet1.cell(1, column = 2).value = '리뷰'

    sheet2 = wb.create_sheet('여름')
    sheet2.cell(1, column = 1).value = '날짜'
    sheet2.cell(1, column = 2).value = '리뷰'

    sheet3 = wb.create_sheet('가을')
    sheet3.cell(1, column = 1).value = '날짜'
    sheet3.cell(1, column = 2).value = '리뷰'

    sheet4 = wb.create_sheet('겨울')
    sheet4.cell(1, column = 1).value = '날짜'
    sheet4.cell(1, column = 2).value = '리뷰'
    
    
    #음식점 이름으로 엑셀 이름 저장
    ret_name = soup.find('title').text.split()[0]
    file_name = ret_name + '.xlsx'

    sp_index = 2
    su_index = 2
    fa_index = 2
    wi_index = 2
    rn = int(rn)
    
    ntime = datetime.now()
    date_list=[]
    for i in range(len(date)):
        if date[i].text[1:3] == '일전':
            a = int(date[i].text[0])
            temp = str(ntime + timedelta(days=-a))
            date_list.append(temp[0:10])
        else:
            date_list.append(date[i].text)
        

    for row_index in range(0, rn):
        try:
            if date_list[row_index][5:7] in spring:
                sheet1.cell(sp_index, column = 1).value = date_list[row_index]
                sheet1.cell(sp_index, column = 2).value = review[row_index].text.strip()  
                sp_index += 1
            elif date_list[row_index][5:7] in summer:   
                sheet2.cell(su_index, column = 1).value = date_list[row_index]
                sheet2.cell(su_index, column = 2).value = review[row_index].text.strip()  
                su_index += 1
            elif date_list[row_index][5:7] in fall:   
                sheet3.cell(fa_index, column = 1).value = date_list[row_index]
                sheet3.cell(fa_index, column = 2).value = review[row_index].text.strip()  
                fa_index += 1
            elif date_list[row_index][5:7] in winter:   
                sheet4.cell(wi_index, column = 1).value = date_list[row_index]
                sheet4.cell(wi_index, column = 2).value = review[row_index].text.strip()      
                wi_index += 1
        except:
            pass
            

    
    wb.save(filename=file_name)


    


## 시험본 _ 연습코드

In [27]:
driver.get(alink[48])
#리뷰수
reviewn = driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/div/ul/li[1]/button').text
rnum = re.findall("\d+", reviewn)
try:
    rn = rnum[len(rnum)-1]
except IndexError as e:
    rn = rnum
    pass

print(rn)
    #===============================================================================
    #더보기

n = int(rn)/5
i=0
try:
    add_show = driver.find_element_by_xpath('/html/body/main/article/div[1]/div[1]/div/section[3]/button')
    for i in range(int(n)):
        add_show.click()
        time.sleep(PAUSE_TIME)
except:
    pass
    
time.sleep(PAUSE_TIME)
text = driver.page_source
soup = BS(text, 'html.parser')
date = soup.find_all('span', 'past-time ng-binding')
review = soup.findAll('span', 'short_review more_review_bind review_content')

43


In [28]:
index = 0
index1 = 0
for i in review:
    index +=1
for i in date:
    index1 += 1

print(index, index1)


5 43


In [29]:
wb = Workbook()

spring = ['03','04','05']
summer = ['06', '07','08']
fall = ['09', '10', '11']
winter = ['12','01','02']

sheet1 = wb.active
sheet1.title = "봄"
sheet1.cell(1, column = 1).value = '날짜'
sheet1.cell(1, column = 2).value = '리뷰'

sheet2 = wb.create_sheet('여름')
sheet2.cell(1, column = 1).value = '날짜'
sheet2.cell(1, column = 2).value = '리뷰'

sheet3 = wb.create_sheet('가을')
sheet3.cell(1, column = 1).value = '날짜'
sheet3.cell(1, column = 2).value = '리뷰'

sheet4 = wb.create_sheet('겨울')
sheet4.cell(1, column = 1).value = '날짜'
sheet4.cell(1, column = 2).value = '리뷰'

In [30]:
for i in range(len(date)):
    if date[i].text[1:3] == '일전':
        a = int(date[i].text[0])
        temp = str(ntime + timedelta(days=-a))
        date_list.append(temp[0:10])
    else:
        date_list.append(date[i].text)

In [31]:
#음식점 이름으로 엑셀 이름 저장
ret_name = soup.find('title').text.split()[0]
file_name = ret_name + '.xlsx'

sp_index = 2
su_index = 2
fa_index = 2
wi_index = 2
rn = int(rn)

ntime = datetime.now()
date_list=[]
for i in range(len(date)):
    if date[i].text[1:3] == '일전':
        a = int(date[i].text[0])
        temp = str(ntime + timedelta(days=-a))
        date_list.append(temp[0:10])
    else:
        date_list.append(date[i].text)
        
try:
    for row_index in range(0, rn):
        if date_list[row_index][5:7] in spring:
            sheet1.cell(sp_index, column = 1).value = date_list[row_index]
            sheet1.cell(sp_index, column = 2).value = review[row_index].text.strip()
            sp_index += 1
        elif date_list[row_index][5:7] in summer:   
            sheet2.cell(su_index, column = 1).value = date_list[row_index]
            sheet2.cell(su_index, column = 2).value = review[row_index].text.strip()
            su_index += 1
        elif date_list[row_index][5:7] in fall:   
            sheet3.cell(fa_index, column = 1).value = date_list[row_index]
            sheet3.cell(fa_index, column = 2).value = review[row_index].text.strip()
            fa_index += 1
        elif date_list[row_index][5:7] in winter:   
            sheet4.cell(wi_index, column = 1).value = date_list[row_index]
            sheet4.cell(wi_index, column = 2).value = review[row_index].text.strip()    
            wi_index += 1
except:
    pass
    
wb.save(filename=file_name)
